The purpose of this notebook is to combine data from our 2 data sources.

Specifically look at the following 2 webpages, with different data on the same election:
- https://www.electionsireland.org/result.cfm?election=1977&cons=219
- https://www.irelandelection.com/election.php?elecid=11&electype=1&constitid=48

Note: The quotas are different.

In [76]:
import pandas as pd
import numpy as np

### Introducing the first data set

The first dataset comes from https://www.electionsireland.org/ 

This data has date 12 columns:

- date: _str_, This is sometimes the exact date of the election, othertimes its just the year, other times its the year and the month: needs to be combined with the other data set which has the year.

- election_type: _str_, Is the type of election the candidate was running in, eg Local, Seanad, Dail, Bi-Election, This data needs to cleaned because it also includes intra-party elections and resignations. (Interesting data but not needed for this analysis)

- party: _str_, The party a candidate ran with, or for independents, it lists them as belonging to a party called independent.

- status: _str_, If they were elected or not. This data needs to be cleaned as it includes other information such as whether the candidate made the cutoff for expenses. 

- constituency_name: _str_, The name of the constituency the candidate ran in.

- seat: _int_, Contains the order in which the candidate was elected, if the candidate wasnt elected then its None.

- count_eliminated: _int_, Which count the candidate was either **elected** or **lost**. 

- first_pref_count _int_, The count of first preference votes recieved. 

- first_pref_pct: _float_, The % of all first preference votes a candidate recieved. 

- pct_of_quota_reached_with_first_pref: _float_, The % of the quota that a candidate reached with first preference votes.

- ran_unopposed: _bool_, True if the candidate ran unopposed (more common in earlier elections)

- candidate: _str_, The candidate's name

- candidate_ID: _int_, A unique ID for each candidate 


In [77]:
df1 = pd.read_parquet('electionsireland_data/ElectionsIreland_candidate.parquet')
df1 = df1.rename(columns={'ID':'candidate_ID'})
print(df1.columns)
print(df1.shape)
df1.head()

Index(['date', 'election_type', 'party', 'status', 'constituency_name', 'seat',
       'count_eliminated', 'first_pref_count', 'first_pref_pct',
       'pct_of_quota_reached_with_first_pref', 'ran_unopposed', 'candidate',
       'candidate_ID'],
      dtype='object')
(30070, 13)


,date,election_type,party,status,constituency_name,seat,count_eliminated,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID
0,23 June 1960,By Election,Labour,Not Elected,Carlow Kilkenny,None,NaN,7678.0,0.2016,NaN,False,Seamus Pattison,1
1,1961,17th Dail,Labour,Elected,Carlow Kilkenny,4,NaN,4116.0,0.0954,NaN,False,Seamus Pattison,1
2,1965,18th Dail,Labour,Elected,Carlow Kilkenny,3,NaN,6299.0,0.1408,NaN,False,Seamus Pattison,1
3,1969,19th Dail,Labour,Elected,Carlow Kilkenny,4,NaN,6041.0,0.1311,NaN,False,Seamus Pattison,1
4,1973,20th Dail,Labour,Elected,Carlow Kilkenny,4,NaN,5300.0,0.1134,NaN,False,Seamus Pattison,1


### Introducing the second dataset

The first dataset comes from https://www.irelandelection.com/ 

This data has date 10 columns:

- election: _str_, Contains a string describing the election usually just the consitituency and the date of the election

- elected: _bool_, If they were elected or not. This data needs to be combined with Status from the first dataframe. 

- party: _str_, The party a candidate ran with

- first_pref_pct: _float_, The % of all first preference votes a candidate recieved. 

- first_pref_count _int_, The count of first preference votes recieved. 

- first_pref_quota_ratio: _float_, The % of the quota that a candidate reached with first preference votes.

- year: _int_, The year of the election. *NB:* Sometimes there is two elections of the same kind in the same year, (eg 2 dail elections in 1982).

- election_type: _str_, Is the type of election the candidate was running in, eg Local, Seanad, Dail, Bi-Election.

- candidate: _str_, The candidate's name. Unfortunatly there is 2 candidates have the same, they are essentially grouped as the same person, we fix this by linking records with the first dataset. 

- constituency: _str_, The name of the constituency the candidate ran in.

- election_type: _str_, Is the type of election the candidate was running in, eg Local, Dail, Bi-Election.

In [78]:
df2 = pd.read_parquet('irelandelection/ALL_CANDIDATES.parquet')
print(df2.columns)
print(df2.shape)

df2.head()

Index(['election', 'elected', 'party', 'first_pref_pct', 'first_pref_count',
       'first_pref_quota_ratio', 'year', 'candidate', 'constituency',
       'election_type'],
      dtype='object')
(36243, 10)


,election,elected,party,first_pref_pct,first_pref_count,first_pref_quota_ratio,year,candidate,constituency,election_type
0,2004 Local Election - Thomastown,True,Labour Party,0.085,641,0.51,2004,Ann Phelan,Thomastown,LOCAL
1,2009 Local Election - Thomastown,True,Labour Party,0.156,1183,0.78,2009,Ann Phelan,Thomastown,LOCAL
2,2011 general election - Carlow–Kilkenny,True,Labour Party,0.109,8072,0.66,2011,Ann Phelan,Carlow–Kilkenny,GENERAL
3,2016 general election - Carlow–Kilkenny,False,Labour Party,0.063,4391,0.38,2016,Ann Phelan,Carlow–Kilkenny,GENERAL
0,1982 (Feb) general election - Carlow–Kilkenny,False,Fianna Fáil,0.017,907,0.10,1982,John McGuinness,Carlow–Kilkenny,GENERAL


### Cleaning Dataframe 1

So we need to clean the following columns in dataframe 1:
- seat and count_eliminated can be dropped as for this analysis we dont really care about the order in which candidates were elected. 
- date, this needs to be converted to a year columns.
- election_type, this should read the same as dataframe 2 with options like GENERAL, LOCAL and SEANAD
- status, will be kept, but we want a column that is a simple bool TRUE or FALSE was the candidate elected.


In [79]:
# dropping seat and count_eliminated

df1 = df1.drop(columns=['seat','count_eliminated'])
df1.head(3)

,date,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID
0,23 June 1960,By Election,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1
1,1961,17th Dail,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1
2,1965,18th Dail,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1


In [80]:
#cleaning date

def get_year_from_date_string(date_str):
    if date_str == None:
        return 0
    elif len(date_str) > 4:
        try:
            return int(date_str[-4:])
        except:
            if isinstance(date_str[-1],int):#last letter is an int
                return date_str[-4:]
    else:
        return int(date_str)

df1 = df1.reset_index().drop(columns=['index'])
df1['year'] = df1.date.apply(get_year_from_date_string)
df1 = df1.drop(columns=['date'])

df1.head(3)

,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID,year
0,By Election,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1,1960.0
1,17th Dail,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1,1961.0
2,18th Dail,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1,1965.0


In [81]:
#cleaning election_type

def get_election_type_from_string(election_type_str):
    if 'Town' in election_type_str or 'Local' in election_type_str :
        return 'LOCAL'
    elif 'Dail' in election_type_str:
        return 'GENERAL'
    elif 'Seanad' in election_type_str:
        return 'SEANAD'
    elif 'Westminster' in election_type_str:
        return 'Westminster'.upper()
    elif 'European' in election_type_str:
        return 'EUROPEAN'
    elif 'By Election' in election_type_str:
        return 'BI-ELECTION'
    else: # for the rows that represent resignations or appointments or some other event in a politicans career
        return None

df1['election_type'] = df1.election_type.apply(get_election_type_from_string)
print(df1.election_type.unique())

['BI-ELECTION' 'GENERAL' 'EUROPEAN' None 'LOCAL' 'SEANAD' 'WESTMINSTER']


In [82]:
df1.status.unique()

array(['Not Elected', 'Elected', 'Appointed', 'Resigned', 'Disqualified',
       None, 'Co-opted', 'Candidate', 'Dublin', 'Lucan',
       'awaiting update', 'changed\xa0to',
       'Dublin University (Trinity College)', 'Died in office:',
       'Agricultural Panel', 'Gorey\xa0\xa0\xa0-\xa0\xa0\xa0Resigned',
       'Nominated by Taoiseach', 'Inishowen', '(Replaced  David McKenna)',
       '(Replaced  Michael Flynn)', 'Administrative Panel', 'Drumcliff',
       'Ballybrack\xa0\xa0\xa0-\xa0\xa0\xa0Resigned\n            \n              (ill health)',
       'Kilkenny\xa0\xa0\xa0-\xa0\xa0\xa0Resigned\n            \n              (dual mandate TD)'],
      dtype=object)

In [83]:
# cleaning status
def was_elected(status):
    status = str(status) # some of the status are None 

    if status == 'Elected':
        return True
    elif status == 'Not Elected':
        return False

    else: # for the rows that represent resignations or appointments or some other event in a politicans career
        return None

df1['elected'] = df1.status.apply(was_elected)
df1.head()

,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID,year,elected
0,BI-ELECTION,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1,1960.0,False
1,GENERAL,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1,1961.0,True
2,GENERAL,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1,1965.0,True
3,GENERAL,Labour,Elected,Carlow Kilkenny,6041.0,0.1311,NaN,False,Seamus Pattison,1,1969.0,True
4,GENERAL,Labour,Elected,Carlow Kilkenny,5300.0,0.1134,NaN,False,Seamus Pattison,1,1973.0,True


### Cleaing dataframe 2

Renaming:
Dataframe 2 has a column called ```first_pref_quota_ratio``` which is the same information as ```pct_of_quota_reached_with_first_pref``` in dataframe 1

In [84]:
df2 = df2.rename(columns={'first_pref_quota_ratio':'pct_of_quota_reached_with_first_pref','constituency':'constituency_name'}).reset_index().drop(columns=['index'])

print(df2.columns)
print(df2.shape)

Index(['election', 'elected', 'party', 'first_pref_pct', 'first_pref_count',
       'pct_of_quota_reached_with_first_pref', 'year', 'candidate',
       'constituency_name', 'election_type'],
      dtype='object')
(36243, 10)


In [85]:
df2[df2.constituency_name.str.contains('Cork South')]

,election,elected,party,first_pref_pct,first_pref_count,pct_of_quota_reached_with_first_pref,year,candidate,constituency_name,election_type
328,1985 Local Election - Cork South,False,Fianna Fáil,0.083,978,0.58,1985,Joe O'Reilly,Cork South,LOCAL
932,1937 general election - Cork South–East,True,Fianna Fáil,0.244,7567,0.98,1937,Martin Corry,Cork South–East,GENERAL
933,1938 general election - Cork South–East,True,Fianna Fáil,0.242,7701,0.97,1938,Martin Corry,Cork South–East,GENERAL
934,1943 general election - Cork South–East,True,Fianna Fáil,0.267,8767,1.07,1943,Martin Corry,Cork South–East,GENERAL
935,1944 general election - Cork South–East,True,Fianna Fáil,0.265,8038,1.06,1944,Martin Corry,Cork South–East,GENERAL
...,...,...,...,...,...,...,...,...,...,...
35487,2020 General Election - Cork South–Central,False,Independent,0.002,102,0.01,2020,Seán O'Leary,Cork South–Central,GENERAL
35495,2020 General Election - Cork South–Central,False,Independent,0.002,103,0.01,2020,John Donohoe,Cork South–Central,GENERAL
35496,2020 General Election - Cork South–Central,False,Solidarity - People Before Profit,0.013,764,0.07,2020,Bobby Murray Walsh,Cork South–Central,GENERAL
35497,2020 General Election - Cork South–Central,False,Independent,0.004,202,0.02,2020,William O'Brien,Cork South–Central,GENERAL


### Fixing Constitenuecy names.

I noticed that the constituency names are slightly different

Examples: 

Some are just slightly different spelling.

In [86]:
df2.loc[(df2.candidate == 'Brian McEnery')]

,election,elected,party,first_pref_pct,first_pref_count,pct_of_quota_reached_with_first_pref,year,candidate,constituency_name,election_type
1223,1994 by-election - Cork South–Central,False,Independent,0.005,219,0.01,1994,Brian McEnery,Cork South–Central,BI-ELECTION
1224,1998 by-election - Cork South–Central,False,Natural Law,0.003,150,0.01,1998,Brian McEnery,Cork South–Central,BI-ELECTION


In [87]:
df1_non_null_constituencies = df1[~df1.constituency_name.isnull()]  # Note: for df1 some of the constituencies are null values because the row isnt representing an election rather the row is data about some other event in a politicians career.

df1_non_null_constituencies.loc[(df1_non_null_constituencies.candidate == 'Brian McEnery')]

,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID,year,elected
30068,BI-ELECTION,Non party/Independent,Not Elected,Cork South Central,219.0,0.0052,0.01,False,Brian McEnery,4211,1994.0,False
30069,BI-ELECTION,Natural Law,Not Elected,Cork South Central,150.0,0.0035,0.01,False,Brian McEnery,4211,1998.0,False


Some are incorretly spelled:

In [88]:
df1_non_null_constituencies.loc[(df1_non_null_constituencies.constituency_name.str.contains('Offaly'))]
# this should read Laois Offaly

,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID,year,elected
17,GENERAL,Republican,Elected,Leix Offaly,5572.0,0.1382,NaN,False,John McGuinness,3,1923.0,True
1566,GENERAL,Non party/Independent,Not Elected,Leix Offaly,1445.0,0.0358,NaN,False,Patrick Belton,600,1923.0,False
1904,None,Fianna Fail,Not Elected,Laois Offaly,4073.0,0.0587,0.35,False,Peter Ormond,747,2020.0,False
1910,GENERAL,Fianna Fail,Not Elected,Laoighis Offaly,4328.0,0.0745,0.45,False,Ger Killally,751,1997.0,False
1912,GENERAL,Fianna Fail,Not Elected,Laoighis Offaly,4719.0,0.0746,0.45,False,Ger Killally,751,2002.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
29912,GENERAL,Non party/Independent,Not Elected,Laoighis Offaly,236.0,0.0037,0.02,False,John Kelly,7773,2002.0,False
30056,BI-ELECTION,Labour,Not Elected,Leix Offaly,9187.0,0.2230,0.45,False,John F Gill,1602,1926.0,False
30057,GENERAL,Labour,Elected,Leix Offaly,3730.0,0.0865,NaN,False,John F Gill,1602,1927.0,True
30058,GENERAL,Labour,Not Elected,Leix Offaly,2962.0,0.0679,NaN,False,John F Gill,1602,1927.0,False


We should check to see if there are some are different names for the same election, I'm pretty sure I saw one example of it but can't think of what it was just now. 

#### Missing data

Next we have a ton of constituency names that are in one dataframe but not the other. This is expected as one dataframe might have election data that the other doesnt have. 

we shouldnt have this many mismatches. 

In [89]:
set(df1[df1['election_type'] =='LOCAL'].constituency_name.unique()).difference(set(df2[df2['election_type'] =='LOCAL'].constituency_name.unique()))
# I dont bother to fix this but we probably should get a list of all constituency names and the map them onto whatever the website calls them. 
# this is easy with dail elections but not so easy with local elections as i cant find anything about it

{'Adare Rathkeale',
 'Artane Whitehall',
 'Athenry Oranmore',
 'Bailieborough Cootehill',
 'Ballaghadereen',
 'Ballincollig Carrigaline',
 'Ballybay',
 'Ballybay Clones',
 'Ballyfermot Drimnagh',
 'Ballymote Tobercurry',
 'Ballymun Finglas',
 'Ballymun Whitehall',
 'Ballyshannon',
 'Bandon Kinsale',
 'Bantry West Cork',
 'Beaumont Donaghmede',
 'Belturbet TC',
 'Blanchardstown Mulhuddart',
 'Blarney Macroom',
 'Borris-in-Ossory Mountmellick',
 'Bray No 1',
 'Bray No 2',
 'Bray No 3',
 'Bray South',
 'Bundoran',
 'Cabra Finglas',
 'Cabra Glasnevin',
 'Callan Thomastown',
 'Cappaghmore Kilmallock',
 'Carlow No 1',
 'Carlow No 2',
 'Carrick on Suir',
 'Carrick-On-Shannon',
 'Carrickmacross Castleblaney',
 'Cashel Tipperary',
 'Cavan Belturbet',
 'Celbridge Leixlip',
 'Clare West',
 'Clonakilty',
 'Conamara',
 'Cootehill',
 'Cork East',
 'Cork West',
 'Crumlin Kimmage',
 'Drogheda No 1 Laurence Gate',
 'Drogheda No 2 West Gate',
 'Drogheda No 3 Duleek Gate',
 'Droichead Nua',
 'Dromahaire'

In [90]:
set(df1[df1['election_type'] =='GENERAL'].constituency_name.unique()).difference(set(df2[df2['election_type'] =='GENERAL'].constituency_name.unique()))

{'Antrim East',
 'Antrim Mid',
 'Antrim North',
 'Antrim South',
 'Armagh Mid',
 'Armagh North',
 'Armagh South',
 'Athlone Longford',
 'Belfast Cromac',
 'Belfast Duncairn',
 'Belfast Falls',
 'Belfast Ormeau',
 'Belfast Pottinger',
 'Belfast Shankill',
 "Belfast St Anne's",
 'Belfast Victoria',
 'Belfast Woodvale',
 'Carlow',
 'Carlow Kildare',
 'Carlow Kilkenny',
 'Cavan East',
 'Cavan Monaghan',
 'Cavan West',
 'Clare East',
 'Clare Galway South',
 'Clare West',
 'Cork',
 'Cork City North',
 'Cork City South',
 'Cork East and North East',
 'Cork Mid/North/South/South East and West',
 'Cork North Central',
 'Cork North East',
 'Cork North West',
 'Cork South Central',
 'Cork South East',
 'Cork South West',
 'Donegal Leitrim',
 'Donegal North',
 'Donegal North East',
 'Donegal South',
 'Donegal South West',
 'Down East',
 'Down Mid',
 'Down North',
 'Down South',
 'Down West',
 'Dublin College Green',
 'Dublin Harbour',
 'Dublin Mid',
 'Dublin Mid West',
 'Dublin North Central',
 'D

#### Scraping down the constituency names 

In [91]:
historic_constituencies = pd.read_html('https://en.wikipedia.org/wiki/Historic_D%C3%A1il_constituencies',flavor='bs4')[1]
historic_constituencies

,Constituency,County or city,Created,Abolished,Seats
0,Antrim,Antrim,1921,1922.0,7
1,Antrim East,Antrim,1918,1921.0,1
2,Antrim Mid,Antrim,1918,1921.0,1
3,Antrim North,Antrim,1918,1921.0,1
4,Antrim South,Antrim,1918,1921.0,1
...,...,...,...,...,...
255,Wexford North,Wexford,1918,1921.0,1
256,Wexford South,Wexford,1918,1921.0,1
257,Wicklow[aq],Wicklow,1923,NaN,345
258,Wicklow East,Wicklow,1918,1921.0,1


In [92]:
current_constituencies = pd.read_html('https://en.wikipedia.org/wiki/D%C3%A1il_constituencies',flavor='bs4')[3]
current_constituencies

,Constituency,Seats
0,Carlow–Kilkenny,5
1,Cavan–Monaghan,5
2,Clare,4
3,Cork East,4
4,Cork North-Central,4
5,Cork North-West,3
6,Cork South-Central,4
7,Cork South-West,3
8,Donegal,5
9,Dublin Bay North,5


What I need you to do is the following:
- Rename the constituencies so that they are uniform across both data sets. 
- Create a JSON that lists a current constiteuncy and its historic constituencies it was related to, dont worry if that list is non exclusive.